In [27]:
import os, sys
import tensorflow as tf
import urllib.request

def nsfw(url):
    urllib.request.urlretrieve(url, "local-filename.jpg")
    image_path = 'local-filename.jpg'
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

    # change this as you see fit
    # image_path = sys.argv[1]
    
    # Read in the image_data
    image_data = tf.gfile.FastGFile(image_path, 'rb').read()

    # Loads label file, strips off carriage return
    label_lines = [line.rstrip() for line 
                       in tf.gfile.GFile("../scripts/NSFW-detection/retrained_labels.txt")]

    # Unpersists graph from file
    with tf.gfile.FastGFile("../scripts/NSFW-detection/retrained_graph.pb", 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def, name='')

    with tf.Session() as sess:
        # Feed the image_data as input to the graph and get first prediction
        softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')

        predictions = sess.run(softmax_tensor, \
                 {'DecodeJpeg/contents:0': image_data})

        return predictions[0][1]

In [30]:
url = 'https://pbs.twimg.com/profile_images/969111536244613120/Jw1uZbBb_400x400.jpg'

In [31]:
nsfw(url)

0.8709387